In [ ]:
from __future__ import print_function, division   # Python 2/3 compatibility
from skimage import io                            # utilities to read and write images in various formats
import numpy as np  # array manipulation package
import matplotlib.pylab as plt                    # plotting package
%matplotlib inline
plt.rcParams['figure.figsize'] = (7,15)         # set default figure size
plt.rcParams['image.cmap'] = 'gray'               # set default colormap to gray

# Digital Image Processing - Programming Assignment 

The following progamming assignment involves image filtering in the frequency domain. The deadline for returning your work is **14 April 2022 at 23:59. 
Please, follow carefully the submission instructions given in the end of this notebook.** You are encouraged to seek information in other places than the course book and lecture material but remember **list all your sources under references**.

If you experience problems that you cannot solve using the course material or the Python documentation, or have any questions regarding to the programming assignments, please do not hesitate to contact the course assistant by e-mail at the address dip@unioulu.oulu.fi.

# 4. Image transforms : lowpass and highpass filtering in frequency domain

In the following, you will first perform ideal lowpass and highpass filtering on the test image, and later, we will also consider the Gaussian lowpass and highpass filtering. First, read the part concerning image enhancement in frequency domain in the lecture notes or in the course book. Specifically, you should look at the **Chapter-4** (available as a PDF file) in the lecture notes in Moodle.

Now, perform the following operations in the reserved code cells and answer to the questions written in bold into the reserved spaces.


**4.1. Read and display the test image `hplptest.jpg`.**

In [ ]:
# read test image
test_img = io.imread('hplptest.jpg')

# display the test image
plt.imshow(test_img)
plt.title(f"Test image")
plt.show()

**4.2. Compute the Fourier transform (FT) of the test image and take a look at what the magnitude of the FT looks like.**

Hint: When plotting the FTs, use logarithmic graylevel transformation to make the result more illustrative for human visual system: 

`>>> np.log(np.abs(image_fft)+1)`

In [ ]:
from scipy import fftpack

# compute the FT of the test image using 'fftpack.fft2'
img_fft = fftpack.fft2(test_img)

# translate the origin of the FT (low frequencies) to the center using 'fftpack.fftshift'
fft_translate = fftpack.fftshift(img_fft)

# display the magnitude of the uncentered and centered FT using 'imshow'.
fig, ax = plt.subplots(figsize=(16,16), nrows=1, ncols=2)
ax[0].imshow(np.log(np.abs(img_fft)+1))
ax[0].axis('off')
ax[0].set_title(f"Uncentered FT")
ax[1].imshow(np.log(np.abs(fft_translate)+1))
ax[1].axis('off')
ax[1].set_title(f"Centered FT")


fig.tight_layout()

**The code for constructing an ideal lowpass filter is given below:**

In [ ]:
# make two frequency matrices, 'f1' and 'f2', as help variables (frequencies from -1 to 1)
n = (500,500)
f1 = ( np.arange(0,n[0])-np.floor(n[0]/2) ) * (2./(n[0]))
f2 = ( np.arange(0,n[1])-np.floor(n[1]/2) ) * (2./(n[1]))
f1, f2 = np.meshgrid(f1, f2)

# make a matrix with absolute values of frequency (“sampled” frequency domain)
D = np.sqrt(f1**2 + f2**2)

# set cut-off frequency D0 to 0.2
D0 = 0.2;

# filter matrix is initialized to ones 
Hlp = np.ones(n)

# set frequencies in filter mask Hlp greater than the cut-off frequency D0 to zero, other elements remain unaltered
Hlp[D>D0] = 0.0

**4.3. Modify the lowpass filter code and construct ideal highpass filter `Hhp` with the same cut-off frequency `D0=0.2` and display both ideal lowpass and highpass filter masks in the same figure.**

In [ ]:
# create ideal highpass filter mask Hhp
# make two frequency matrices, 'f1' and 'f2', as help variables (frequencies from -1 to 1)
n = (500,500)
f1 = ( np.arange(0,n[0])-np.floor(n[0]/2) ) * (2./(n[0]))
f2 = ( np.arange(0,n[1])-np.floor(n[1]/2) ) * (2./(n[1]))
f1, f2 = np.meshgrid(f1, f2)

# make a matrix with absolute values of frequency (“sampled” frequency domain)
D = np.sqrt(f1**2 + f2**2)

# set cut-off frequency D0 to 0.2
D0 = 0.2;

# filter matrix is initialized to ones 
Hhp = np.ones(n)

# set frequencies in filter mask Hhp smaller than the cut-off frequency D0 to zero, other elements remain unaltered
Hhp[D<D0] = 0.0

# display the filters
fig, ax = plt.subplots(figsize=(16,16), nrows=1, ncols=2)
ax[0].imshow(Hlp)
ax[0].axis('off')
ax[0].set_title(f"Low-pass filter")
ax[1].imshow(Hhp)
ax[1].axis('off')
ax[1].set_title(f"High-pass filter")
fig.tight_layout()

**4.4. Perform ideal lowpass and highpass filtering in the frequency domain by multiplying the centralized FT of the original image with the `Hlp` and `Hhp` filter masks (element-per-element matrix multiplication) and display the two resulting FTs in the same figure.**

In [ ]:
# apply ideal lowpass and highpass filtering to the test image, i.e. multiply element-wise the fft of the image with the filter masks
lo_filt = fft_translate*Hlp
hi_filt = fft_translate*Hhp
# display the magnitude of the resulting FTs
fig, ax = plt.subplots(figsize=(16,16), nrows=1, ncols=2)
ax[0].imshow(np.log(np.abs(lo_filt)+1))
ax[0].axis('off')
ax[0].set_title(f"Low-pass filtered FT")
ax[1].imshow(np.log(np.abs(hi_filt)+1))
ax[1].axis('off')
ax[1].set_title(f"High-pass filtered FT")
fig.tight_layout()


**4.5. Reconstruct the filtered images with `fftpack.ifft2()` and `fftpack.ifftshift()` in reverse order and display the two filtered images using `imshow()` in the same figure.** 

Hint: Due to round-off errors, you have to take the real part of the result of inverse FT before displaying it with `imshow()`. Please note also that the resulting images values beyond the original `uint8` image `[0,255]`, so you need to clip these values using `np.clip()`.

In [ ]:
# reconstruct the filtered images
filtImLoPass_fft = fftpack.ifftshift(lo_filt)
filtImHiPass_fft = fftpack.ifftshift(hi_filt)
filtImLoPass = fftpack.ifft2(filtImLoPass_fft)
filtImHiPass = fftpack.ifft2(filtImHiPass_fft)

# take the 'real' part of the resulting images due to possible round-off errors
lopass_im = np.real(filtImLoPass)
hipass_im = np.real(filtImHiPass)

# clip values beyond the uint8 range [0,255]
lopass_im = np.clip(lopass_im, 0, 255)
hipass_im = np.clip(hipass_im, 0 , 255)

# display the original image and its lowpass and highpass filtered images in the same figure
fig, ax = plt.subplots(figsize=(16,16), nrows=1, ncols=3)
ax[0].imshow(test_img)
ax[0].axis('off')
ax[0].set_title(f"Original test image")
ax[1].imshow(lopass_im)
ax[1].axis('off')
ax[1].set_title(f"Low-pass filtered test image")
ax[2].imshow(hipass_im)
ax[2].axis('off')
ax[2].set_title(f"High-pass filtered test image")
fig.tight_layout()


**4.6. Now, construct Gaussian lowpass and highpass filters with cut-off frequency `D0=0.2` and display them in the same figure.**

Hint: All you need to do is to modify the filter matrix `Hlp` line in the example code snippet accordingly to form `Hlpg` and `Hhpg` (see, formula 4.3-7 in the course book or the lecture notes, specifically, the **chapter04.pdf**).

In [ ]:
# construct Gaussian lowpass and highpass filters

# Gaussian lowpass filter construction
# make two frequency matrices, 'f1' and 'f2', as help variables (frequencies from -1 to 1)
n = (500,500)
f1 = ( np.arange(0,n[0])-np.floor(n[0]/2) ) * (2./(n[0]))
f2 = ( np.arange(0,n[1])-np.floor(n[1]/2) ) * (2./(n[1]))
f1, f2 = np.meshgrid(f1, f2)

# make a matrix with absolute values of frequency (“sampled” frequency domain)
D = np.sqrt(f1**2 + f2**2)

# set cut-off frequency D0 to 0.2
D0 = 0.2;

# initialize filter matrix
Hlpg = np.exp(-(D**2) / (2*D0))


#-------------------------------------------------------------------------------------------s
# Gaussian highpass filter construction
# make two frequency matrices, 'f1' and 'f2', as help variables (frequencies from -1 to 1)
n = (500,500)
f1 = ( np.arange(0,n[0])-np.floor(n[0]/2) ) * (2./(n[0]))
f2 = ( np.arange(0,n[1])-np.floor(n[1]/2) ) * (2./(n[1]))
f1, f2 = np.meshgrid(f1, f2)

# make a matrix with absolute values of frequency (“sampled” frequency domain)
D = np.sqrt(f1**2 + f2**2)

# set cut-off frequency D0 to 0.2
D0 = 0.2;

# initialize filter matrix
Hhpg = 1 - np.exp(-(D**2) / (2*D0))


# display the filter masks
fig, ax = plt.subplots(figsize=(16,16), nrows=1, ncols=2)
ax[0].imshow(Hlpg)
ax[0].axis('off')
ax[0].set_title(f"Gaussian lowpass filter")
ax[1].imshow(Hhpg)
ax[1].axis('off')
ax[1].set_title(f"Gaussian highpass filter")

**4.7. Perform Gaussian lowpass and highpass filtering to the original test image and display the magnitude of the resulting FTs in the same figure.**

In [ ]:
# apply gaussian lowpass and highpass filtering to the test image
gauss_lo = fft_translate*Hlpg
gauss_hi = fft_translate*Hhpg

# display the magnitude of the resulting FTs
fig, ax = plt.subplots(figsize=(16,16), nrows=1, ncols=2)
ax[0].imshow(np.log(np.abs(gauss_lo)+1))
ax[0].axis('off')
ax[0].set_title(f"Gaussian lowpass filtered FT")
ax[1].imshow(np.log(np.abs(gauss_hi)+1))
ax[1].axis('off')
ax[1].set_title(f"Gaussian highpass filtered FT")

**4.8. Finally, reconstruct the filtered images like in step 4.5. and display the original image and the two Gaussian filtered images in the same figure.**

In [ ]:
# reconstruct the filtered images 
filtImLoPassGauss_fft = fftpack.ifftshift(gauss_lo)
filtImHiPassGauss_fft = fftpack.ifftshift(gauss_hi)
filtImLoPassGauss = fftpack.ifft2(filtImLoPassGauss_fft)
filtImHiPassGauss = fftpack.ifft2(filtImHiPassGauss_fft)

# take the 'real' part of the resulting images due to possible round-off errors
lopass_gauss = np.real(filtImLoPassGauss)
hipass_gauss = np.real(filtImHiPassGauss)

# clip values beyond the uint8 range [0,255]
lopass_gauss = np.clip(lopass_gauss, 0, 255)
hipass_gauss = np.clip(hipass_gauss, 0 , 255)

# display the three images in the same figure
fig, ax = plt.subplots(figsize=(16,16), nrows=1, ncols=3)
ax[0].imshow(test_img)
ax[0].axis('off')
ax[0].set_title(f"Original test image")
ax[1].imshow(lopass_gauss)
ax[1].axis('off')
ax[1].set_title(f"Gaussian lowpass filtered image")
ax[2].imshow(hipass_gauss)
ax[2].axis('off')
ax[2].set_title(f"Gaussian highpass filtered image")

# Aftermath
Finally, fill your answers to the following questions:

# References
`Enter your references here`

# Submission

1. Before submitting your work, **check that your notebook (code) runs from scratch** and reproduces all the requested results by clicking on the menu `Kernel -> Restart & Run All`! Also, check that you have answered all the questions written in **bold**.
2. Clear all outputs and variables, etc. by click on the menu `Kernel -> Restart & Clear Output`. This may (or will) reduce the file size of your deliverable a lot! 
3. Rename this Jupyter notebook to **`DIP_PA3_[student number(s)].ipynb`** (e.g. `DIP_PA3_1234567.ipynb` if solo work or `DIP_PA3_1234567-7654321.ipynb` if pair work) and upload it as your submission to Moodle.